In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import torch
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

# !pip install torchvision
import torchvision

import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

#!pip install torchmetrics
import torchmetrics

### Download the dataset for CNN

In [6]:
#!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
#!unzip data.zip

In [7]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Model
For this homework we will use Convolutional Neural Network (CNN). We'll use PyTorch.

You need to develop the model with following structure:

1. The shape for input should be (3, 200, 200) (channels first format in PyTorch)
2. Next, create a convolutional layer (nn.Conv2d):
- Use 32 filters (output channels)
- Kernel size should be (3, 3) (that's the size of the filter)
- Use 'relu' as activation
3. Reduce the size of the feature map with max pooling (nn.MaxPool2d)
- Set the pooling size to (2, 2)
4. Turn the multi-dimensional result into vectors using flatten or view
5. Next, add a nn.Linear layer with 64 neurons and 'relu' activation
6. Finally, create the nn.Linear layer with 1 neuron - this will be the output
7. The output layer should have an activation - use the appropriate activation for the binary classification case
8. As optimizer use torch.optim.SGD with the following parameters:
```
torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
```

In [8]:
class CNN(nn.Module):
   def __init__(self):

       """
       CNN model for binary classification.
       Parameters:
           * in_channels: Number of channels in the input image (for grayscale images, 1)
           * num_classes: Number of classes to predict. In our problem, 10 (i.e digits from  0 to 9).

       Architecture:
       - Conv2d layer with 32 filters, kernel size (3,3), ReLU
       - MaxPool 2d layer size (2,2)
       - Flatten
       - Linear -> 64, ReLU
       - Linear -> 1, Sigmoid
       """
       super(CNN, self).__init__()

       # 1. the shape of the input should be (3,200,200)
       # 1st convolutional layer
       self.conv1 = nn.Conv2d(
           in_channels=3,
           out_channels=32,
           kernel_size=3,
           padding=1        # same padding -> output stays 200x200
           )
       # 2. Max pooling layer
       self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

       # After Conv + Pool:
       # Input: (3,200,200)
       # Conv1 : (32,200,200)
       # Pool: (32,100,100)

       # Flatten or view Next
       flattened_size = 32*100*100

       # 3. First fully connected layer (64 neurons)
       self.fc1 = nn.Linear(flattened_size, 64)

       # 4. Final output linear layer: 1 neuron
       self.fc2 = nn.Linear(64,1)

       # Sigmoid activation for probability output
       self.sigmoid = nn.Sigmoid()

   def forward(self, x):
       """
       Define the forward pass of the neural network.

       Parameters:
           x: Input tensor.

       Returns:
           torch.Tensor
               The output tensor after passing through the network.
       """
       x = F.relu(self.conv1(x))  # Apply first convolution and ReLU activation
       x = self.pool(x)           # Apply max pooling
       x = x.reshape(x.shape[0], -1)  # Flatten the tensor
       x = self.fc1(x)            # Apply first fully connected layer
       x = F.relu(x)           # ReLU
       x = self.fc2(x)                    # Dense Output Layer
       x = self.sigmoid(x)                # Sigmoid activation for binary classification
       return x

# Optimizer
model = CNN()
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

In [9]:
print(model)

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=320000, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [14]:
# create some random test data (batch size = 4)
x_test = torch.randn(4,3,200,200)
y_test_binary = torch.randint(0,2,(4,1)).float() # for BCE/MSE
y_test_class = torch.randint(0,10,(4,1))  # for CrossEntropy
y_test_cos = torch.randn(4,1)  # for CosineEmbedding

# Loss functions to test
criterions_dict = {
    "MSELoss": nn.MSELoss(),
    "BCEWithLogitsLoss": nn.BCEWithLogitsLoss(),
    "CrossEntropyLoss": nn.CrossEntropyLoss(),
    "CosineEmbeddingLoss": nn.CosineEmbeddingLoss()
}

print("\n=== Testing loss functions ===")

for name, criterion in criterions_dict.items():
    print(f"\n{name}")
    try:
        # Forward pass
        outputs = model(x_test)

        if name == "CrossEntropyLoss":
            # CE needs logits of shape (batch, num_classes)
            # model outputs (batch, 1) → will ERROR (this is expected)
            loss = criterion(outputs, y_test_class)

        elif name == "CosineEmbeddingLoss":
            # Cosine requires same shape + target of +1 / -1
            loss = criterion(outputs, torch.randn_like(outputs), y_test_cos)

        else:
            # MSELoss, BCEWithLogitsLoss
            loss = criterion(outputs, y_test_binary)

        print("Loss:", loss.item())

    except Exception as e:
        print("Error:", e)




=== Testing loss functions ===

MSELoss
Loss: 0.2633136212825775

BCEWithLogitsLoss
Loss: 0.7338293194770813

CrossEntropyLoss
Error: Expected floating point type for target with class probabilities, got Long

CosineEmbeddingLoss
Error: 0D or 1D target tensor expected, multi-target not supported


In [17]:
# Q2 What's the total number of parameters of the model?

# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
sm = summary(model, input_size=(3, 200, 200))
print(sm)

print("--------------### Alternative ###-------------")
# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 200, 200]             896
         MaxPool2d-2         [-1, 32, 100, 100]               0
            Linear-3                   [-1, 64]      20,480,064
            Linear-4                    [-1, 1]              65
           Sigmoid-5                    [-1, 1]               0
Total params: 20,481,025
Trainable params: 20,481,025
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 12.21
Params size (MB): 78.13
Estimated Total Size (MB): 90.79
----------------------------------------------------------------
None
--------------### Alternative ###-------------
Total parameters: 20481025


In [22]:
# generators and training

# To transform both train and tests sets
train_transforms = transforms.Compose(
    [
        transforms.Resize((200,200)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )  # ImageNet normalization
    ]
)


device = 'cuda' if torch.cuda.is_available() else 'cpu'

# -----------------------
# Train DataLoader
# -----------------------
# Locate Images dir
train_dataset = datasets.ImageFolder(
    root='data/train',
    transform = train_transforms
)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers = 2
)


# -----------------------
# Val DataLoader
# -----------------------
validation_dataset = datasets.ImageFolder(
    root='data/test',
    transform = train_transforms
)

validation_loader = DataLoader(
    validation_dataset,
    batch_size=32,
    shuffle=False,
    num_workers = 2
)


num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}
criterion = nn.BCEWithLogitsLoss()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")


Epoch 1/10, Loss: 0.6843, Acc: 0.4875, Val Loss: 0.6668, Val Acc: 0.4876
Epoch 2/10, Loss: 0.6579, Acc: 0.4875, Val Loss: 0.6555, Val Acc: 0.4876
Epoch 3/10, Loss: 0.6465, Acc: 0.4875, Val Loss: 0.6555, Val Acc: 0.4876
Epoch 4/10, Loss: 0.6397, Acc: 0.4875, Val Loss: 0.6560, Val Acc: 0.4876
Epoch 5/10, Loss: 0.6312, Acc: 0.4875, Val Loss: 0.6594, Val Acc: 0.4876
Epoch 6/10, Loss: 0.6282, Acc: 0.4875, Val Loss: 0.6683, Val Acc: 0.4876
Epoch 7/10, Loss: 0.6209, Acc: 0.4875, Val Loss: 0.6593, Val Acc: 0.4876
Epoch 8/10, Loss: 0.6165, Acc: 0.4875, Val Loss: 0.6593, Val Acc: 0.4876
Epoch 9/10, Loss: 0.6114, Acc: 0.4875, Val Loss: 0.6605, Val Acc: 0.4876
Epoch 10/10, Loss: 0.6067, Acc: 0.4875, Val Loss: 0.6669, Val Acc: 0.4876


In [26]:
### Data Augmentation

# part2. To add transformations for both train and test separately
train_transforms = transforms.Compose([
    transforms.RandomRotation(50),                    # rotate up to ±50 degrees
    transforms.RandomResizedCrop(
        200,
        scale=(0.9, 1.0),
        ratio=(0.9, 1.1)
    ),                                               # random crop + resize to 200x200
    transforms.RandomHorizontalFlip(),               # flip left-right
    transforms.ToTensor(),                           # convert image → tensor (0–1)
    transforms.Normalize([0.485, 0.456, 0.406],      # normalize (ImageNet mean/std)
                         [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


device = 'cuda' if torch.cuda.is_available() else 'cpu'

# -----------------------
# Train DataLoader
# -----------------------
# Locate Images dir
train_dataset = datasets.ImageFolder(
    root='data/train',
    transform = train_transforms
)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers = 2
)


# -----------------------
# Val DataLoader
# -----------------------
validation_dataset = datasets.ImageFolder(
    root='data/test',
    transform = val_transforms
)

validation_loader = DataLoader(
    validation_dataset,
    batch_size=32,
    shuffle=False,
    num_workers = 2
)

# running epochs

num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}
criterion = nn.BCEWithLogitsLoss()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")



Epoch 1/10, Loss: 0.6336, Acc: 0.4900, Val Loss: 0.6469, Val Acc: 0.4975
Epoch 2/10, Loss: 0.6371, Acc: 0.4913, Val Loss: 0.6468, Val Acc: 0.4975
Epoch 3/10, Loss: 0.6343, Acc: 0.4900, Val Loss: 0.6484, Val Acc: 0.4975
Epoch 4/10, Loss: 0.6264, Acc: 0.4925, Val Loss: 0.6512, Val Acc: 0.4975
Epoch 5/10, Loss: 0.6353, Acc: 0.4900, Val Loss: 0.6486, Val Acc: 0.5025
Epoch 6/10, Loss: 0.6382, Acc: 0.4938, Val Loss: 0.6464, Val Acc: 0.4975
Epoch 7/10, Loss: 0.6289, Acc: 0.4913, Val Loss: 0.6499, Val Acc: 0.5025
Epoch 8/10, Loss: 0.6301, Acc: 0.4938, Val Loss: 0.6481, Val Acc: 0.4975
Epoch 9/10, Loss: 0.6295, Acc: 0.4913, Val Loss: 0.6514, Val Acc: 0.4975
Epoch 10/10, Loss: 0.6320, Acc: 0.4925, Val Loss: 0.6469, Val Acc: 0.4975
